In [1]:
import numpy as np

import math as m

from pulp import *

import pandas as pd

In [2]:
minmax = int(input("Min press 1 or Max press 2 :"))

Min press 1 or Max press 2 :1


In [3]:
dim = int(input("Enter M: ")) 

din = int(input("Enter N: "))

 

################################# input matrix A ################################

 

print("Enter Matrix A :")                           

Ac = []              #center of matrix A

Au = []              #upper bound of matrix A 

Al = []               #lower nound of matrix A

delta = []           #radius of matrix A

A = []               #matrix A

for m in range(dim):

    Ac_1 = []

    Au_1 = []
    
    Al_1 = []

    delta_1 = []
    
    a_1 = []

    for n in range(din):
        
        a_2 = []

        value_A = input("Enter 2 numbers (with a comma in between): ")                  #input matrix A in the pattern of (a, b)

        a, b = value_A.split(',')                                                       #splits a and b

        a, b = float(a), float(b)                                                       #transfrom str(a), str(b) to float(a), float(b) respectively

        Ac_1.append((a+b)*0.5)                                                                                      

        Au_1.append(b)                                                                  #add b(upper bound) in upper bound matrix
        
        Al_1.append(a)                                                                   #add a(lower bound) in lower bound matrix

        delta_1.append((b-a)*0.5)
        
        a_2.append(a)
        
        a_2.append(b)
        
        a_1.append(a_2)

    Ac.append(Ac_1)

    Au.append(Au_1)
    
    Al.append(Al_1)

    delta.append(delta_1)
    
    A.append(a_1)

print("Matrix A is", A)   

print("Center matrix of A =", Ac)

print("Lower bound matrix of A =", Al)

print("Upper bound matrix of A =", Au)

print("Radius matrix of A =", delta, '\n')

 

################################# input matrix B ################################

 

print("Enter Matrix B :")

Bc = []         #center of matrix b

Bl = []         #lower bound of matrix b

Bu = []        #uppper bound of matrix b

tuda = []       #radius of matrix b

B = []          #matrix B

for m in range(dim):
    
    b_1 = []
    
    value_B = input("Enter 2 numbers (with a comma in between): ")

    c, d = value_B.split(',')

    c, d = float(c), float(d)

    Bc.append((c+d)*0.5)

    Bl.append(c)
    
    Bu.append(d)

    tuda.append((d-c)*0.5)
    
    b_1.append(c)
    
    b_1.append(d)
    
    B.append(b_1)
   
print("Matrix B is", B)
    
print("Center matrix of B =", Bc)

print("Lower bound matrix of B =", Bl)

print("Upper bound matrix of B =", Bu)

print('Radius matrix of B =', tuda, '\n')

 

################################# input matrix C ################################

 

print("Enter Matrix C :")

C = []             #matrix c

for r in range(din):

    value_C = float(input("Enter number: "))

    C.append(value_C)

   

print("Matrix C =", C, '\n')






if minmax ==1 :
    model = LpProblem("Minimax regret of minimization interval linear programming problem", LpMinimize)
else:
    model = LpProblem("Minimax regret of maximization interval linear programming problem", LpMinimize)

variable_names = [str(i) for i in range(1, din+1)]

variable_names.sort()




DV_variables = LpVariable.matrix("X", variable_names, lowBound= 0 )

X = np.transpose(np.array(DV_variables).reshape(1, din))

print("Decision Variable/Allocation Matrix: ")

print(np.transpose(X))




R = LpVariable.matrix("R", str(1), lowBound = 0)




obj = np.array([1])

model.setObjective(obj @ R)

Enter M: 2
Enter N: 4
Enter Matrix A :
Enter 2 numbers (with a comma in between): 4,6
Enter 2 numbers (with a comma in between): 5,7
Enter 2 numbers (with a comma in between): -8,-6
Enter 2 numbers (with a comma in between): 7,9
Enter 2 numbers (with a comma in between): 9,11
Enter 2 numbers (with a comma in between): -12,-10
Enter 2 numbers (with a comma in between): 11,13
Enter 2 numbers (with a comma in between): 12,14
Matrix A is [[[4.0, 6.0], [5.0, 7.0], [-8.0, -6.0], [7.0, 9.0]], [[9.0, 11.0], [-12.0, -10.0], [11.0, 13.0], [12.0, 14.0]]]
Center matrix of A = [[5.0, 6.0, -7.0, 8.0], [10.0, -11.0, 12.0, 13.0]]
Lower bound matrix of A = [[4.0, 5.0, -8.0, 7.0], [9.0, -12.0, 11.0, 12.0]]
Upper bound matrix of A = [[6.0, 7.0, -6.0, 9.0], [11.0, -10.0, 13.0, 14.0]]
Radius matrix of A = [[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0]] 

Enter Matrix B :
Enter 2 numbers (with a comma in between): -10,-8
Enter 2 numbers (with a comma in between): 13,15
Matrix B is [[-10.0, -8.0], [13.0, 15.0]]

In [4]:
################################# Minimization interval LP ########################################

if minmax == 1:


#### Opti Min ####    
    
    opti_min = LpProblem("Optimistic obj. value fn of minimization interval LP", LpMinimize)
    obj_func_opti_min = C @ X
    opti_min.setObjective(obj_func_opti_min[0])
    
    for m in range(dim):
        opti_min += lpSum(Au[m][i] * X[i][0] for i in range(dim)) >=  Bl[m]
    print(opti_min)
    
    opti_min.solve(PULP_CBC_CMD())
    status_opti_min =  LpStatus[opti_min.status]   
    print('Status: ',status_opti_min)
    if status_opti_min == 'Optimal':
        print("Optimistic obj. value fn of minimization interval LP is:", opti_min.objective.value(), '\n')
    
        for w in opti_min.variables():
            try:
                print(w.name,"=", w.value())
            except:
                print("error couldnt find value")
                
            
    print('________________________________________________________________________________________________________________', '\n')
     
 #### Pess Min #####     
    pess_min = LpProblem("Pessimistic obj. value fn of minimization interval LP", LpMinimize)
    obj_func_pess_min = C @ X
    pess_min.setObjective(obj_func_pess_min[0])
    
    for m in range(dim):
        pess_min += lpSum(Al[m][i] * X[i][0] for i in range(dim)) >=  Bu[m]
    print(pess_min)
    
    pess_min.solve(PULP_CBC_CMD())
    status_pess_min =  LpStatus[pess_min.status]   
    print('Status: ',status_pess_min)
    if status_opti_min == 'Optimal':
        print("Pessimistic obj. value fn of minimization interval LP is:", pess_min.objective.value())
        
        for v in pess_min.variables():
            try:
                print(v.name,"=", v.value())
            except:
                print("error couldnt find value")
            
    print('________________________________________________________________________________________________________________', '\n')
       
    
#### Minimax regret Min ####          
            
    if LpStatus[pess_min.status] == 'Optimal' :
    
        for m in range(dim):
            model += Bu[m] <=  lpSum(Al[m][i] * X[i][0] for i in range(dim))
        model += ((pess_min.objective.value() - C@X)[0] <= R )
        print(model)
    
        model.solve(PULP_CBC_CMD())
        status =  LpStatus[model.status]    
        print(status)    
        print("Minimax regret is:", model.objective.value())    
        
        for v in model.variables():
            try:
                print(v.name,"=", v.value())
            except:
                print("error couldnt find value")
            
    else:
        print("This interval linear programming does not have minimax regret.")
            
            
################################## Maxization interval LP #######################################

elif minmax == 2:
    
#### Opti Max ####    
    
    opti_max = LpProblem("Optimistic obj. value fn of maximization interval LP", LpMaximize)
    obj_func_opti_max = C @ X
    opti_max.setObjective(obj_func_opti_max[0])
    
    for m in range(dim):
        opti_max += lpSum(Al[m][i] * X[i][0] for i in range(dim)) <=  Bu[m]
    print(opti_max)
    
    opti_max.solve(PULP_CBC_CMD())
    status_opti_max =  LpStatus[opti_max.status]   
    print('Status', status_opti_max)  
    if status_opti_max == 'Optimal':
        print("Optimistic obj. value fn of this maximization interval LP is:", opti_max.objective.value(), '\n')
        
        for q in opti_max.variables():
            try:
                print(q.name,"=", q.value())
            except:
                print("error couldnt find value")
            
    print('________________________________________________________________________________________________________________', '\n')
    
    

#### Pess Max #####    

    pess_max = LpProblem("Pessimistic obj. value fn of maximization interval LP", LpMaximize)
    obj_func_pess_max = C @ X
    pess_max.setObjective(obj_func_pess_max[0])
    
    for m in range(dim):
        pess_max += lpSum(Au[m][i] * X[i][0] for i in range(dim)) <=  Bl[m]
    print(pess_max)
    
    pess_max.solve(PULP_CBC_CMD())
    status_pess_max =  LpStatus[pess_max.status]   
    print('Status', status_pess_max)   
    if status_pess_max == 'Optimal':
        print("Pessimistic obj. value fn of this maximization interval LP is:", pess_max.objective.value(), '\n')
        
        for z in pess_max.variables():
            try:
                print(z.name,"=", z.value())
            except:
                print("error couldnt find value")
            
    print('________________________________________________________________________________________________________________', '\n')    
    
    
            
#### Minimax regret Max #### 

    b = pess_max.objective.value()
            
    if LpStatus[pess_max.status] == 'Optimal' :
    
        for m in range(dim):
            model += lpSum(Au[m][i] * X[i][0] for i in range(dim)) <= Bl[m]
        model += ((b - C@X)[0] <= R )
        print(model)
    
        model.solve(PULP_CBC_CMD())
        status =  LpStatus[model.status]    
        print(status)    
        print("Minimax regret is:", model.objective.value())
        
        for v in model.variables():
            try:
                print(v.name,"=", v.value())
            except:
                print("error couldnt find value")
    else:
        print("This interval linear programming does not have minimax regret.")

Optimistic obj. value fn of minimization interval LP:
MINIMIZE
-1.0*X_1 + -2.0*X_2 + 3.0*X_3 + 4.0*X_4 + 0.0
SUBJECT TO
_C1: 6 X_1 + 7 X_2 >= -10

_C2: 11 X_1 - 10 X_2 >= 13

VARIABLES
X_1 Continuous
X_2 Continuous
X_3 Continuous
X_4 Continuous

Status:  Unbounded
________________________________________________________________________________________________________________ 

Pessimistic obj. value fn of minimization interval LP:
MINIMIZE
-1.0*X_1 + -2.0*X_2 + 3.0*X_3 + 4.0*X_4 + 0.0
SUBJECT TO
_C1: 4 X_1 + 5 X_2 >= -8

_C2: 9 X_1 - 12 X_2 >= 15

VARIABLES
X_1 Continuous
X_2 Continuous
X_3 Continuous
X_4 Continuous

Status:  Unbounded
________________________________________________________________________________________________________________ 

This interval linear programming does not have minimax regret.
